# import



In [ ]:
#環境確認
import pandas as pd
import numpy as np
!python3 --version
print(pd.__version__)
print(np.__version__)
import matplotlib
print(matplotlib.__version__)
import pandas as pd
import numpy as np
import random
import matplotlib.pylab as plt
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold,KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn .linear_model import LinearRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
import warnings
import xgboost as xgb
from sklearn.model_selection import KFold
from geopy.distance import geodesic
from tqdm.notebook import tqdm
from statistics import mean
import seaborn as sns
from datetime import datetime

warnings.filterwarnings('ignore')
# from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
#     ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

input_dir = "/content/drive/MyDrive/機械学習/220312_民泊サービスの宿泊料金予測/"

for i in tqdm(range(10)):
    i

In [ ]:
lgb.__version__
!pip install lightgbm==3.3.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# データ読み込み

In [ ]:
#データの読み込み
train_df = pd.read_csv("{}input/train_data.csv".format(input_dir))
test_df = pd.read_csv("{}input/test_data.csv".format(input_dir))
station_df = pd.read_csv("{}input/station_list.csv".format(input_dir))
submit_df = pd.read_csv("{}input/submission.csv".format(input_dir))
print(train_df.shape)
print(test_df.shape)
print(station_df.shape)

# データの理解と特徴量エンジニアリング

In [ ]:
# データの確認
train_df.head()

In [ ]:
# カテゴリデータの確認
print(train_df['neighbourhood'].unique())

### 最寄り駅までの距離を特徴量に追加

In [ ]:
# 重複排除
station_df = station_df.drop_duplicates(subset=['station_name']).reset_index(drop=True)

train_len = len(train_df)
station_len = len(station_df)
station_len

# trainデータに最寄り駅までの距離追加
train_df['ns_distance'] = 0
for i in tqdm(range(train_len)):
    dis = []
    # TODO
    # for j in range(ここに本来のコードを入れる):
    for j in range(649):
        hotel = (train_df['latitude'][i],train_df['longitude'][i])
        station = (station_df['latitude'][j],station_df['longitude'][j])
        dis.append(geodesic(hotel, station).m)
       
        
    train_df['ns_distance'][i] = min(dis)

train_df['ns_distance']

In [ ]:
# testデータに最寄り駅までの距離追加
test_df['ns_distance'] = 0

test_len = len(test_df)

for i in range(test_len):
    dis = []
    for j in range(649):
        hotel = (test_df['latitude'][i],test_df['longitude'][i])
        station = (station_df['latitude'][j],station_df['longitude'][j])
        dis.append(geodesic(hotel, station).m)
    test_df['ns_distance'][i] = min(dis)

test_df['ns_distance']

In [ ]:
# 出力
train_df.to_csv('{}input/train_df_station.csv'.format(input_dir),index=False)
test_df.to_csv('{}input/test_df_station.csv'.format(input_dir),index=False)

In [ ]:
# 入力
train_df = pd.read_csv("{}input/train_df_station.csv".format(input_dir))
test_df = pd.read_csv("{}input/test_df_station.csv".format(input_dir))

### encoding

In [ ]:
# 説明変数と被説明変数の設定
train_columns = ['minimum_nights', 'number_of_reviews', 'availability_365', 'room_type', 'neighbourhood','ns_distance']
target = 'y'

X_train = train_df[train_columns]
y_train = np.log(train_df[target])

X_test = test_df[train_columns]

y_train

In [ ]:
# ダミー変数化
# train_df = pd.get_dummies(train_df, columns=['room_type'], prefix='rt', drop_first=True)
# test_df = pd.get_dummies(test_df, columns=['room_type'], prefix='rt', drop_first=True)


# ラベルエンコーディング

cat_cols = ["room_type", "neighbourhood"]

for c in cat_cols:
  le = LabelEncoder()
  le.fit(X_train[c])
  X_train[c] = le.transform(X_train[c])
  X_test[c] = le.transform(X_test[c])


# frequency encoding

# for c in cat_cols:
#     X_train[c] = X_train.groupby(c)[c].transform('count')
#     X_test[c] = X_test.groupby(c)[c].transform('count')



# ターゲットエンコーディング

# for c in cat_cols:
#   data_tmp = pd.DataFrame({c:X_train[c],'target':y_train})
#   target_mean = data_tmp.groupby(c)['target'].mean()
#   X_test[c] = X_test[c].map(target_mean)
#   tmp = np.repeat(np.nan, X_train.shape[0])
#   kf = KFold(n_splits=4, shuffle=True, random_state=72)
#   for idx_1,idx_2 in kf.split(X_train):
#     target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
#     tmp[idx_2] = X_train[c].iloc[idx_2].map(target_mean)
#     X_train[c] = tmp

X_train.head()

In [ ]:
X_train.describe()

In [ ]:
# 欠損値確認
X_train.isna().sum()

### 特徴量追加

In [ ]:
# 特徴量を追加する関数
def make_feat(df):
  df["neighbourhood"] = df["neighbourhood"]
  df["room_type"] = df["room_type"]
  df["minimum_nights"] = df["minimum_nights"]
  df["number_of_reviews"] = df["number_of_reviews"]
  df["availability_365"] = df["availability_365"]
  df["minimum_nights*neighbourhood"] = df["minimum_nights"]*df["neighbourhood"]
  df["availability_365*neighbourhood"] = df["availability_365"]*df["neighbourhood"]
  df["availability_365**2*neighbourhood**2"] = df["availability_365"]**2*df["neighbourhood"]**2
  df["availability_365/room_type"] = df["availability_365"]/df["room_type"]
  df["neighbourhood**2/room_type"] = df["neighbourhood"]**2/df["room_type"]
  df["availability_365**2/room_type"] = df["availability_365"]**2/df["room_type"]
  df["neighbourhood*room_type**2"] = df["neighbourhood"]*df["room_type"]**2
  df["availability_365**2*minimum_nights**2"] = df["availability_365"]**2*df["minimum_nights"]**2
  df["availability_365**2*number_ofviews**2"] = df["availability_365"]**2*df["number_of_reviews"]**2
  df["sqrt(availability_365)*minimum_nights"] = np.sqrt(df["availability_365"])*df["minimum_nights"]

In [ ]:
# 特徴量の追加
make_feat(X_train)
make_feat(X_test)
X_train.shape,X_test.shape

In [ ]:
#7:3でtrainデータとvalidデータに分割

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# X_train

In [ ]:

categorical_features = {*sorted(['room_type	', 'neighbourhood'])}

# モデルの生成（lightGBM）

In [ ]:
all_param = {
"colsample_bytree": 0.32714832683589756,
"learning_rate": 0.006840905564844016,
"max_bin": 166,
"min_child_samples": 5,
"n_estimators": 573,
"num_leaves": 114,
"subsample": 0.956238192643021,
"subsample_freq": 3,
}

In [ ]:
#LGBMで学習する関数
def build_model(X, y, all_param):
    # 学習設定
    FOLD = 5

    # パラメータセット
    lgb_params = {
      'objective': 'regression',
      'importance_type': 'gain',
      'metric': 'rmse',
      'seed': 42,
      'n_jobs': -1,
      'verbosity': -1,

      'n_estimators': all_param['n_estimators'],
      'learning_rate': all_param['learning_rate'],
      'boosting_type': 'gbdt',
      'subsample': all_param['subsample'],
      'subsample_freq': all_param['subsample_freq'],
      'colsample_bytree': all_param['colsample_bytree'],
      'num_leaves': all_param['num_leaves'],
      'min_child_samples': all_param['min_child_samples'],
      'max_bin': all_param['max_bin'],
      'force_col_wise': True,
    }

    # 返却用リスト
    preds_list = list()
    models_list = list()
    scores_list = list()

    # 学習
    kf = KFold(n_splits=FOLD, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

        model = lgb.LGBMRegressor(**lgb_params)

        model = lgb.LGBMRegressor(**lgb_params)
        model.fit(X_train, y_train,                
                eval_set=(X_valid, y_valid),
                eval_metric='rmse',
                callbacks=[lgb.early_stopping(100),],
                
        )

        y_pred = model.predict(X_valid)
        score = np.sqrt(mean_squared_error(y_valid, y_pred))

        pred = np.zeros(len(X))
        pred[test_index] = y_pred
        preds_list.append(pred)
        scores_list.append(score)
        models_list.append(model)

    return preds_list, scores_list, models_list

In [ ]:
y_train = pd.DataFrame(y_train, columns=['y'])
type(y_train)

In [ ]:
y_train.iloc[100]

In [ ]:
# 学習
# TODO warning確認
preds_list, scores_list, models_list = build_model(X_train, y_train, all_param)

# スコア確認

In [ ]:
print(scores_list)
print('score mean:{}'.format(mean(scores_list)))

df_score = pd.DataFrame(scores_list, columns=['score'])
df_score['count'] = range(1, len(scores_list) + 1)
fig, ax = plt.subplots(figsize=(16, 9))
sns.lineplot(x='count', y='score', data=df_score)

# 予測

In [ ]:
# 予測
pred_list = list()
for model in models_list:
    pred_data = model.predict(X_test)
    # 対数化された予測値を戻す。
    pred_data = np.expm1(pred_data)
    pred_list.append(pred_data)

pred_list

In [ ]:
df_pred = pd.DataFrame(pred_list)
df_pred = pd.DataFrame(df_pred.mean())
df_pred

# OOFの予測を追加

In [ ]:
# 予測
df_pred_oof = pd.DataFrame()
for i, pred in enumerate(preds_list):
    df_pred_oof[f"FOLD_{i}"] = np.expm1(pred)

df_pred_oof["FOLD_mean"] = df_pred_oof.T.sum()
df_pred_oof

# 提出用のファイル作成

In [ ]:
exp = "012"
now = datetime.now().strftime("%y%m%d_%H%M%S")
df_submit = pd.read_csv("{}input/submission.csv".format(input_dir))
df_submit = pd.concat([df_submit, df_pred], axis=1)
df_submit.drop("y", axis=1, inplace=True)
df_submit.columns = ["id", "y"]
df_submit.to_csv('{}output/{}_j_{}_submission.csv'.format(input_dir, now, exp),index=False)
df_pred_oof.to_csv('{}output/{}_j_{}_pred_oof.csv'.format(input_dir, now, exp),index=False)

In [ ]:
# validationデータ出力用
submit_valid_df = pd.read_csv("{}input/submission.csv".format(input_dir))
submit_valid_df['y']=pd.Series(y_pred.reshape(-1,))
submit_valid_df.to_csv('{}output/submission（y_valid）.csv'.format(input_dir),index=False)

In [ ]:
# 提出用
submit_df = pd.read_csv("{}input/submission.csv".format(input_dir))
submit_df['y']=pd.Series(pred_data.reshape(-1,))
submit_df.to_csv('{}output/submission（station7）.csv'.format(input_dir),index=False)